In [10]:
import torch
import optuna
import numpy as np
from mlearn import base
from tqdm import tqdm, trange
from mlearn.utils.metrics import Metrics
from mlearn.modeling.embedding import MLPClassifier
from mlearn.data.clean import Cleaner, Preprocessors
from jsonargparse import ArgumentParser, ActionConfigFile
from torchtext.data import TabularDataset, Field, LabelField, BucketIterator
from mlearn.utils.train import train_singletask_model
from mlearn.utils.pipeline import process_and_batch, param_selection

In [2]:
class TorchTextDefaultExtractor:
    """A class to get index-tensor batches from torchtext data object."""

    def __init__(self, datafield: str, labelfield: str, dataloader: base.DataType):
        """Initialize batch generator for torchtext."""
        self.data, self.df, self.lf = dataloader, datafield, labelfield

    def __len__(self):
        """Get length of the batches."""
        return len(self.data)

    def __iter__(self):
        """Iterate over batches in the data."""
        for batch in self.data:
            X = getattr(batch, self.df)
            y = getattr(batch, self.lf)
            yield (X, y)

In [3]:
# Initialize experiment
datadir = 'data/'
torch.random.manual_seed(42)
np.random.seed(42)
encoding = 'index'
tokenizer = 'ekphrasis'
metrics = ['f1-score', 'precision', 'recall', 'accuracy']
display_metric = stop_metric = 'f1-score'
batch_size=  64
epochs = 50
learning_rate = 0.001
dropout = 0.0
embedding = 100
hidden = 100
nonlinearity = 'relu'
gpu = False
hyperopt = False
save_path = None
train_metrics = Metrics(metrics, display_metric, stop_metric)
dev_metrics = Metrics(metrics, display_metric, stop_metric)
c = Cleaner(['url', 'hashtag', 'username', 'lower'])
experiment = Preprocessors(datadir).select_experiment('word')
onehot = True if encoding == 'onehot' else False

if tokenizer == 'spacy':
    tokenizer = c.tokenize
elif tokenizer == 'bpe':
    tokenizer = c.bpe_tokenize
elif tokenizer == 'ekphrasis':
    tokenizer = c.ekphrasis_tokenize

# Set annotations, corrections and filters.
annotate = {'elongated', 'emphasis'}
filters = [f"<{filtr}>" for filtr in annotate]

c._load_ekphrasis(annotate, filters)

/Users/zeerakw/.virtualenvs/mtl/lib/python3.8/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...


/Users/zeerakw/.virtualenvs/mtl/lib/python3.8/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [43]:
text = Field(tokenize = tokenizer, lower = True, batch_first = True)
label = LabelField()
fields = [('ignore', None), ('text', text), ('label', label), ('ignore', None)]
train, dev, test = TabularDataset.splits('/Users/zeerakw/PhD/projects/active/MTL_abuse/data/', train = 'wulczyn_train.tsv',
                                 validation = 'wulczyn_dev.tsv', test = 'wulczyn_test.tsv', 
                                 format = 'tsv', skip_header = True, fields = fields)
text.build_vocab(train)
label.build_vocab(train)

In [5]:
model = MLPClassifier(len(text.vocab.stoi), embedding, hidden, len(label.vocab.stoi), False, nonlinearity)
loss = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

In [6]:
train_ds, dev_ds = BucketIterator.splits(datasets = (train, dev), batch_size = batch_size)
batched_train = TorchTextDefaultExtractor('text', 'label', train_ds)
batched_dev = TorchTextDefaultExtractor('text', 'label', dev_ds)

/Users/zeerakw/.virtualenvs/mtl/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [47]:
foo = next(iter(train_ds))
#print([text.vocab.itos[token] for token in foo.text[0]])
bar = []
baz = []
for document in train_ds:
    for i, doc in enumerate(document.text):
        baz.append(label.vocab.itos[document.label[i]])
        toks = []
        for tok in doc:
            token = text.vocab.itos[tok]
            if token == '<pad>':
                break
            toks.append(token)
        bar.append(toks)

In [50]:
print(len(baz))
print(len(bar))

95692
95692


In [11]:
train_singletask_model(model, save_path, epochs, batched_train, loss, optimizer, train_metrics, batched_dev, dev_metrics, shuffle = False, gpu = False)

Batch:   7%|▋         | 106/1496 [01:00<12:39,  1.83it/s, batch_loss=0.0044]


Batch:  14%|█▍        | 212/1496 [01:49<09:51,  2.17it/s, batch_loss=0.0127]


Batch:  21%|██▏       | 318/1496 [02:36<08:33,  2.29it/s, batch_loss=0.0067]


Batch:  28%|██▊       | 424/1496 [03:25<08:49,  2.02it/s, batch_loss=0.0051]


Batch:  35%|███▌      | 530/1496 [04:13<07:36,  2.11it/s, batch_loss=0.0044]


...

Batch:  99%|█████████▉| 1480/1496 [12:24<00:09,  1.77it/s, batch_loss=0.0065]


Batch: 100%|██████████| 1496/1496 [12:33<00:00,  1.88it/s, batch_loss=0.0051]
                                                                             
Batch:   6%|▌         | 89/1496 [00:51<13:30,  1.74it/s, batch_loss=0.0044]


Batch:  13%|█▎        | 195/1496 [01:58<13:54,  1.56it/s, batch_loss=0.0057]


Batch:  18%|█▊        | 268/1496 [02:40<12:08,  1.69it/s, batch_loss=0.0057]


KeyboardInterrupt: 